# Category Theory for Programmers

## Chapter 2: Types and Functions

### Challenge 5
There are $|\text{Bool}|^{|\text{Bool}|^1} = 2^2 = 4$ functions from `Bool` to `Bool`.

In [ ]:
f1 :: Bool -> Bool
f1 = id

f2 :: Bool -> Bool
f2 = not

f3 :: Bool -> Bool
f3 = const False

f4 :: Bool -> Bool
f4 = const True

## Chapter 3: Categories Great and Small

### Challenge 3

```haskell
import Data.Monoid

instance Monoid Bool where
    mempty = True
    mappend = (&&)

instance Monoid Bool where
    mempty = False
    mappend = (||)
```

### Challenge 5

In [ ]:
import Data.Monoid
import Data.Semigroup

newtype Z3 = Z3 Int
    deriving Show

instance Semigroup Z3 where
    (Z3 x) <> (Z3 y) = Z3 ((x + y) `mod` 3)

instance Monoid Z3 where
    mempty = Z3 0

In [ ]:
foldMap Z3 [2, 1, 2]

## Chapter 6: Simple Algebraic Data Types

### Challenge 1
Show the isomorphism between `Maybe a` and `Either () a`.

In [ ]:
maybeToEither :: Maybe a -> Either () a
maybeToEither Nothing = Left ()
maybeToEither (Just x) = Right x

eitherToMaybe :: Either () a -> Maybe a
eitherToMaybe (Left _) = Nothing
eitherToMaybe (Right x) = Just x

In [ ]:
eitherToMaybe $ maybeToEither Nothing

In [ ]:
eitherToMaybe $ maybeToEither $ Just 42

### Challenge 4
Define new shape `Square` and extend `area` and `circ`.

In [ ]:
data Shape = Circle Float | Rect Float Float | Square Float

area :: Shape -> Float
area (Circle r) = pi * r^2
area (Rect d h) = d * h
area (Square s) = s^2

circ :: Shape -> Float
circ (Circle r) = 2 * pi * r
circ (Rect d h) = 2 * (d + h)
circ (Square s) = 4 * s

### Challenge 5
Show that $a + a = 2 \times a$ up to isomorphism where $a \times a$ is `(a, b)` and $a + b$ is `Either a b` ($2$ in this setup corresponds to `Bool`).

The definition below works up to isomorphism in the choice between
 1. `Left -> False` vs `Left -> True`
 1. `(Bool, a)` vs `(a, Bool)`

In [ ]:
toFlagged :: Either a a -> (Bool, a)
toFlagged (Left x) = (False, x)
toFlagged (Right x) = (True, x)

fromFlagged :: (Bool, a) -> Either a a
fromFlagged (False, x) = Left x
fromFlagged (True, x) = Right x

## Chapter 7: Functors

### Challenge 1
Can the following definition for `Maybe` be made into a `Functor`?
```haskell
fmap :: (a -> b) -> Maybe a -> Maybe b
fmap _ _ = Nothing
```

#### Id Law: $\text{fmap}_f \text{id}_a = \text{id}_{f a}$

- Case `Nothing`:
```haskell
fmap id Nothing
= Nothing
= id Nothing
```
- Case `Just` is not satisfied!
```haskell
fmap id (Just x)
= Nothing
/= Just x
= id (Just x)
```

### Challenge 2
Prove the `Functor` laws for the *Reader* functor:
```haskell
instance Functor ((->) r) where
    -- fmap :: (a -> b) -> (r -> a) -> (r -> b)
    fmap f g = f . g
```

#### Id Law: `fmap id = id`
```haskell
fmap id f
= id . f
= f
```
So `fmap id` has the same effect on `f` as if `f` was just alone, i.e. the same as `id` has. Therefore `fmap id = id`. 

#### Composition Law: `fmap (g . f) = fmap g . fmap f`
Case `Just` is not satisfied!
```haskell
fmap (g . f) h
= (g . f) . h
= g . f . h
= g . (f . h)
= fmap g (f . h)
= fmap g (fmap f h)
= (fmap g . fmap f) h
```

### Challenge 4
Prove the functor laws for the list functor.

```haskell
instance Functor [] where
    -- fmap :: (a -> b) -> [a] -> [b]
    fmap _ [] = []
    fmap f (x:xs) = f x : (fmap f xs)
```

#### Id Law: `fmap id = id`
Base case `[]`:
```haskell
fmap id []
= []
```
i.e. `fmap id` acts as `id` on `[]`.

For the inductive step, let's assume that the relation `fmap id xs = xs` holds for some list `xs`. Then
```haskell
fmap id (x:xs)
= {- applying the definition of fmap -}
  id x : (fmap id xs)
= {- applying id on x -}
  x : (fmap id xs)
= {- induction hypothesis -}
  x : xs
```
proves that `fmap id` acts on `(x:xs)` just like `id`.

#### Composition Law: `fmap (g . f) = fmap g . fmap f`
Base case `[]`:
```haskell
fmap (g . f) []
= []
= fmap g []
= fmap g (fmap f [])
= (fmap g . fmap f) []
```

For the inductive step, let's assume that the relation `fmap (g . f) xs = (fmap g . fmap f) xs` holds for some list `xs`. Then
```haskell
fmap (g . f) (x:xs)
= {- applying the definition of fmap -}
  (g . f) x : (fmap (g . f) xs)
= {- applying (g . f) on x -}
  g (f x) : (fmap (g . f) xs)
= {- induction hypothesis -}
  g (f x) : ((fmap g . fmap f) xs)
= {- writing down (fmap g . fmap f) explicitly -}
  g (f x) : (fmap g (fmap f xs))
= {- factoring out g mapping the head (f x) and the tail (fmap f xs) as (fmap g) over the cons operator -}
  fmap g (f x : (fmap f xs))
= {- repeat previous step for f -}
  fmap g (fmap f (x:xs))
= {- rewrite double fmap application as a composition (fmap g . fmap f) -}
  (fmap g . fmap f) (x:xs)
```
proves that `fmap (g . f)` acts on `(x:xs)` just like `(fmap g . fmap f)`.